# Madrid Air Quality

Die folgenden Daten müssen von kaggle.com kopiert, entpackt und dann in Databricks geladen werden:
- air-quality-madrid.zip                   151 MB
- weather_madrid_LEMD_1997_2015.csv.zip    148 KB

## Analysieren der Wetter Daten von Madrid der Jahre 1997 - 2015

In [1]:
weather = sqlContext.read.format("com.databricks.spark.csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("/FileStore/tables/Madrid/weather_madrid_LEMD_1997_2015.csv")

NameError: name 'sqlContext' is not defined

In [ ]:
weather.printSchema()

In [ ]:
weather.show()

In [ ]:
weather.registerTempTable("weatherT")

In [ ]:
res1 = sqlContext.sql("SELECT CET, `Max TemperatureC`, `Min TemperatureC`, `Mean TemperatureC` FROM weatherT")

In [ ]:
res1.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.clf()
ax = plt.gca()
pdDF = res1.toPandas()
pdDF.plot(x='CET', y='Max TemperatureC', ax=ax)
pdDF.plot(x='CET', y='Min TemperatureC', ax=ax)
pdDF.plot(x='CET', y='Mean TemperatureC', ax=ax)
display()

## Analysieren der Daten der Luftqualität von Madrid

## Messstationen

Einlesen der Daten der Messstationen aus CSV-File:

In [ ]:
stations = sqlContext.read.format("com.databricks.spark.csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("/FileStore/tables/Madrid/stations.csv")
stations.registerTempTable("stationsT")

In [ ]:
stationsRes = sqlContext.sql("SELECT * FROM stationsT")
stationsRes.show()

In [ ]:
stationsRes.count()

## Luftqualitäten pro Jahr

Einlesen der Daten der Luftqualitäten pro Jahr aus CSV-File:

In [ ]:
air_2001 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2001.csv")
air_2002 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2002.csv")
air_2003 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2003.csv")
air_2004 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2004.csv")
air_2005 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2005.csv")
air_2006 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2006.csv")
air_2007 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2007.csv")
air_2008 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2008.csv")
air_2009 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2009.csv")
air_2010 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2010.csv")
air_2011 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2011.csv")
air_2012 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2012.csv")
air_2013 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2013.csv")
air_2014 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2014.csv")
air_2015 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2015.csv")
air_2016 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2016.csv")
air_2017 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2017.csv")
air_2018 = sqlContext.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/Madrid/madrid_2018.csv")


Daten von 2001 ansehen:

In [ ]:
air_2001.registerTempTable("air_2001T")

In [ ]:
res2001 = sqlContext.sql("SELECT date, CO, NO_2, station FROM air_2001T")
res2001.show()

Daten von 2001 der Station 28079007 darstellen:

In [ ]:
res2001S007 = sqlContext.sql("SELECT date, CO, NO_2, station FROM air_2001T WHERE station=28079007")

In [ ]:
plt.clf()
ax = plt.gca()
pdDF = res2001S007.toPandas()
#pdDF.plot(x='date', y='CO', ax=ax)
pdDF.plot(x='date', y='NO_2', ax=ax)
display()

Vereinigung der Jahres-Datentabellen zu einem File.
Dabei werden/müssen einzelne Kolonnen angepasst bzw. entfernt werden.

In [ ]:
air_df1 = air_2001.unionByName(air_2002).unionAll(air_2003)
air_df2 = air_2004.unionByName(air_2005).unionAll(air_2006).unionAll(air_2007).unionAll(air_2008).unionAll(air_2009).unionAll(air_2010)
air_df3 = air_2011.unionByName(air_2012).unionAll(air_2013).unionAll(air_2014).unionAll(air_2015).unionAll(air_2016)
air_df4 = air_2017.unionByName(air_2018)
air_df1.printSchema()
air_df2.printSchema()
air_df3.printSchema()
air_df4.printSchema()

In [ ]:
common_columns = list(set(air_df1.columns).intersection(air_df2.columns).intersection(air_df3.columns).intersection(air_df4.columns))
print(common_columns)

In [ ]:
for column in list(set(air_df1.columns) - set(common_columns)):
  air_df1 = air_df1.drop(column)
air_df1.printSchema()
for column in list(set(air_df2.columns) - set(common_columns)):
  air_df2 = air_df2.drop(column)
air_df2.printSchema()
for column in list(set(air_df3.columns) - set(common_columns)):
  air_df3 = air_df3.drop(column)
air_df3.printSchema()
for column in list(set(air_df4.columns) - set(common_columns)):
  air_df4 = air_df4.drop(column)
air_df4.printSchema()

In [ ]:
air_df = air_df1.unionByName(air_df2).unionByName(air_df3).unionByName(air_df4)
air_df.printSchema()

Anzahl der Messreihen ausgeben:

In [ ]:
#air_df = air_df.dropDuplicates()
air_df.registerTempTable("airT")
res1 = sqlContext.sql("SELECT count(*) FROM airT")
res1.show()

CSV-File schreiben mit allen Messreihen:

In [ ]:
dbutils.fs.rm("/FileStore/tables/Madrid/madrid_2001-2018", True)
air_df.repartition(1).write.csv("/FileStore/tables/Madrid/madrid_2001-2018", sep=",",header = 'true')

Parquet-File mit allen Messreihen schreiben:

In [ ]:
air_df.write.parquet("/FileStore/tables/Madrid/madrid_2001-2018.parquet")

Anpassen des Zeitstempel-Formats:

In [ ]:
air_df_pd = air_df.toPandas()
air_df_pd["red_datetime"] = [air_df_pd["date"][i].replace(hour = 0, minute = 0) for i in range(len(air_df_pd))]
air_df_pd.head(2)

In [ ]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)
air_df_enhanced = sqlCtx.createDataFrame(air_df_pd)

Anpassen der Werte durch Berechnung des Durchschnitts:

In [ ]:
air_df_enhanced.registerTempTable("airT_enh")
time_gas_con = sqlContext.sql("SELECT red_datetime, station, AVG(BEN) AS BEN, AVG(CO) AS CO, AVG(EBE) AS EBE, AVG(NMHC) AS NMHC, AVG(NO_2) AS NO_2, AVG(O_3) AS O_3, AVG(PM10) AS PM10, AVG(SO_2) AS SO_2, AVG(TCH) AS TCH, AVG(TOL) AS TOL FROM airT_enh GROUP BY red_datetime, station")

In [ ]:
time_gas_con.take(10)

Darstellung der Anzahl Messreihen:

In [ ]:
print(time_gas_con.count())

Ausgabe der neuen Tabelle in Parquet-File:

In [ ]:
time_gas_con.write.parquet("/FileStore/tables/Madrid/madrid_2001-2018_time_gas_concentration.parquet")